<a href="https://colab.research.google.com/github/soralh1611/Gen-AI-RAG-and-SFT/blob/main/AI_Powered_Loan_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**

# Project

Soral: Intelligent Underwriting Assistant
SFT-Tuned Gemini with RAG & State-Machine Orchestration
1. Executive Summary
Artha Underwriter 2.0 is an advanced AI agent designed to automate the initial stages of loan eligibility assessment. By combining Supervised Fine-Tuning (SFT) for persona adherence with Retrieval-Augmented Generation (RAG) for factual grounding, the system provides accurate, policy-compliant underwriting decisions in a playful yet professional conversational format.

2. Technical Architecture
The system is built upon three primary pillars of modern LLM engineering:

Supervised Fine-Tuning (SFT): The base Gemini 2.0 model was fine-tuned on a specialized dataset of underwriting scenarios to master the "Soral" persona and internalize complex financial logic.

Retrieval-Augmented Generation (RAG): Real-time integration with institutional policy documents ensures that every decision is anchored in current lending standards, significantly mitigating hallucinations.

LLM Orchestration: A custom state-machine layer manages the conversation flow, ensuring a strict, non-looping 4-step data collection process (Name → ID → FICO → Income).

3. Core Features
Linear Data Gates: Validates user inputs (e.g., 6-digit ID check) and prevents the model from proceeding until current requirements are met.

Anti-Loop Safeguards: Utilizes custom Frequency and Presence Penalties to maintain conversational momentum and prevent repetitive questioning.

Safety & Compliance: Implemented rigorous safety filters to block derogatory language and identify fraudulent input patterns.

4. Notebook Structure
Environment Setup: Library installations and Vertex AI initialization.

Model Configuration: Loading the SFT-tuned model from the Vertex AI Registry.

RAG Integration: Connecting to the knowledge base for grounded decision-making.

Orchestration Logic: Defining the state-machine and generation parameters.

Gradio Interface: Launching the interactive underwriting chatbot.

In [ ]:
from google.cloud import aiplatform

# Define your project variables
PROJECT_ID = "soral-vertex-a"
REGION = "us-central1"
BUCKET_URI = "gs://soral-lms_bucket" # Used for storing model artifacts

# Initialize the Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

print(f"Vertex AI SDK initialized for project {PROJECT_ID}")

Vertex AI SDK initialized for project soral-vertex-a


In [ ]:
from google.colab import auth
auth.authenticate_user()

import vertexai
vertexai.init(project="soral-vertex-a", location="us-central1")

In [ ]:
from google.cloud import storage

def create_new_bucket(bucket_name, location="us-central1"):
    """Creates a new bucket in the specified location."""
    storage_client = storage.Client(project=PROJECT_ID)

    # 1. Clean name (remove gs:// if accidentally added)
    clean_name = bucket_name.replace("gs://", "").lower()

    try:
        # 2. Check if it already exists
        if storage_client.lookup_bucket(clean_name):
            print(f"⚠️ Bucket '{clean_name}' already exists.")
            return storage_client.get_bucket(clean_name)

        # 3. Create the bucket
        bucket = storage_client.create_bucket(clean_name, location=location)

        # 4. Optional: Enable Uniform Bucket-Level Access (Recommended for AI projects)
        bucket.iam_configuration.uniform_bucket_level_access_enabled = True
        bucket.patch()

        print(f"✅ Success: Bucket '{bucket.name}' created in {location}")
        return bucket

    except Exception as e:
        print(f"❌ Error creating bucket: {e}")

# Call the function with a unique name
# Tip: Use your name or project ID as a prefix
NEW_BUCKET_NAME = "lms-reports-soral-2025"
my_bucket = create_new_bucket(NEW_BUCKET_NAME)

⚠️ Bucket 'lms-reports-soral-2025' already exists.


In [ ]:
pip install faker reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.3 MB/s eta 0:00:00


In [ ]:
from faker import Faker
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import random

fake = Faker()

def generate_bank_statement(filename, account_holder):
    doc = SimpleDocTemplate(filename)
    elements = []
    styles = getSampleStyleSheet()

    # Header
    elements.append(Paragraph(f"<b>Bank of Vertex AI - Monthly Statement</b>", styles['Title']))
    elements.append(Paragraph(f"Account Holder: {account_holder}", styles['Normal']))
    elements.append(Paragraph(f"Statement Period: Dec 2025", styles['Normal']))

    # Transaction Data
    data = [["Date", "Description", "Amount", "Balance"]]
    balance = 5000.00
    for _ in range(15):
        date = f"2025-12-{random.randint(1, 20):02d}"
        desc = fake.company()
        amt = round(random.uniform(-500, 1000), 2)
        balance += amt
        data.append([date, desc, f"${amt}", f"${round(balance, 2)}"])

    # Table Styling
    t = Table(data)
    t.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(t)
    doc.build(elements)

generate_bank_statement("bank_statement_demo.pdf", "John Doe")

In [ ]:
pip install faker faker-credit-score reportlab

In [ ]:
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from faker_credit_score import CreditScore
from faker.providers import DynamicProvider
from faker.providers import BaseProvider

fake = Faker()
fake.add_provider(CreditScore)

def generate_credit_report(filename, applicant_name):
    doc = SimpleDocTemplate(filename, pagesize=LETTER)
    styles = getSampleStyleSheet()
    elements = []

    # Custom Style for "Confidential" Header
    header_style = ParagraphStyle('HeaderStyle', parent=styles['Normal'], fontSize=10, textColor=colors.red)

    # 1. Header Section
    elements.append(Paragraph("EQUIFAX - CONFIDENTIAL CONSUMER CREDIT FILE", header_style))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(f"<b>Subject:</b> {applicant_name}", styles['Title']))
    elements.append(Paragraph(f"<b>File Number:</b> {fake.uuid4()}", styles['Normal']))
    elements.append(Paragraph(f"<b>Date of Report:</b> Dec 21, 2025", styles['Normal']))
    elements.append(Spacer(1, 20))

    # 2. Credit Score Section (The "Big Number")
    score = fake.credit_score()
    score_data = [[f"EQUIFAX BEACON 5.0 SCORE: {score}"]]
    score_table = Table(score_data, colWidths=[400])
    score_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, -1), colors.lightgrey),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTSIZE', (0, 0), (-1, -1), 18),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 10),
    ]))
    elements.append(score_table)
    elements.append(Spacer(1, 20))

    # 3. Trade Lines (Credit Accounts)
    elements.append(Paragraph("<b>ACCOUNT HISTORY (TRADE LINES)</b>", styles['Heading2']))
    trade_data = [["Creditor", "Account Type", "Balance", "Status"]]

    # 1. DEFINE the class first
    class BankProvider(BaseProvider):
      def bank_name(self):
          banks = [
              "Chase Bank", "Wells Fargo", "Bank of America",
              "Vertex AI Financial", "Gemini Trust", "Goldman Sachs",
              "PNC Bank", "Citigroup", "Barclays"
          ]
          return self.random_element(banks)
    # 4. Add your custom provider to the Faker instance
    fake.add_provider(BankProvider)
    for _ in range(5):
        trade_data.append([
            fake.bank_name(),
            random.choice(["Revolving", "Installment", "Mortgage"]),
            f"${fake.random_int(0, 15000)}",
            random.choice(["Current", "30 Days Past Due", "Paid as Agreed"])
        ])

    t = Table(trade_data, colWidths=[150, 100, 80, 120])
    t.setStyle(TableStyle([
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke)
    ]))
    elements.append(t)

    doc.build(elements)

    from google.cloud import storage


def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    clean_name = bucket_name.replace("gs://", "")
    bucket = storage_client.get_bucket(clean_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"✅ Success: Uploaded {source_file_name} to {clean_name}")

# --- EXECUTION STEPS ---

# Set your names
MY_BUCKET = "lms-reports-soral-2025"
FILE_NAME = "synthetic_report.pdf"

# STEP 1: Generate the file (Fixes Errno 2)
generate_credit_report(FILE_NAME, "Alex Rivera")

# STEP 2: Now that the file exists, upload it (Fixes 404)
upload_to_gcs(MY_BUCKET, FILE_NAME, "reports/december_report_01.pdf")



✅ Success: Uploaded synthetic_report.pdf to lms-reports-soral-2025


In [ ]:
import os
import random
from faker import Faker
from faker.providers import BaseProvider
from faker_credit_score import CreditScore
from google.cloud import storage
from google.cloud.storage import transfer_manager
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# 1. SETUP
fake = Faker()
class BankProvider(BaseProvider):
    def bank_name(self):
        return self.random_element(["Chase", "Wells Fargo", "Vertex AI Bank", "Gemini Trust"])

fake.add_provider(BankProvider)
fake.add_provider(CreditScore)

BUCKET_NAME = "lms-reports-soral-2025"
LOCAL_DIR = "bulk_data_reports"
os.makedirs(LOCAL_DIR, exist_ok=True)
styles = getSampleStyleSheet()

# 2. DATA GENERATION FUNCTION
def generate_full_report(i):
    name = fake.name()
    u_id = f"{i:04d}"
    filename = os.path.join(LOCAL_DIR, f"report_{u_id}.pdf")

    doc = SimpleDocTemplate(filename)

    # --- CRITICAL: Create a NEW story list for every file ---
    story = []

    # Add Title
    story.append(Paragraph(f"<b>Financial Audit: {name}</b>", styles['Title']))
    story.append(Spacer(1, 12))

    # Add Financial Summary
    summary_data = [
        ["Metric", "Value"],
        ["Credit Score", str(fake.credit_score())],
        ["Monthly Income", f"${random.randint(3000, 12000)}"],
        ["Primary Bank", fake.bank_name()]
    ]
    summary_table = Table(summary_data, colWidths=[150, 150])
    summary_table.setStyle(TableStyle([('BACKGROUND', (0,0), (-1,0), colors.lightgrey), ('GRID', (0,0), (-1,-1), 1, colors.black)]))
    story.append(summary_table)
    story.append(Spacer(1, 20))

    # Add 15 Mock Transactions (Ensures file is NOT empty)
    trans_data = [["Date", "Merchant", "Amount", "Type"]]
    for _ in range(15):
        trans_data.append([
            str(fake.date_this_year()),
            fake.company(),
            f"${random.randint(-1000, 2000)}",
            random.choice(["Debit", "Credit", "ACH"])
        ])

    trans_table = Table(trans_data, colWidths=[80, 150, 80, 80])
    trans_table.setStyle(TableStyle([('GRID', (0,0), (-1,-1), 0.5, colors.grey), ('FONTSIZE', (0,0), (-1,-1), 8)]))
    story.append(trans_table)

    # FINAL STEP: Build PDF
    doc.build(story)
    return filename

# 3. RUN & UPLOAD
def run_bulk_and_upload(count=1000):
    all_filenames = []
    print(f"🛠️ Generating {count} data-rich reports...")
    for i in range(count):
        all_filenames.append(os.path.basename(generate_full_report(i)))
        if i % 100 == 0: print(f"Progress: {i}/{count}")

    print("🚀 Bulk Uploading to GCS...")
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)

    transfer_manager.upload_many_from_filenames(
        bucket,
        all_filenames,
        source_directory=LOCAL_DIR,
        max_workers=8
    )
    print("✅ All 1,000 files uploaded with data.")

run_bulk_and_upload(1000)

🛠️ Generating 1000 data-rich reports...
Progress: 0/1000
Progress: 100/1000
Progress: 200/1000
Progress: 300/1000
Progress: 400/1000
Progress: 500/1000
Progress: 600/1000
Progress: 700/1000
Progress: 800/1000
Progress: 900/1000
🚀 Bulk Uploading to GCS...
✅ All 1,000 files uploaded with data.


In [ ]:
pip install streamlit google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.0 MB/s eta 0:00:00


In [ ]:
!pip install -q streamlit
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠴

In [ ]:
!pip install -q gradio google-genai

In [ ]:
import json
import os
from google.cloud import storage

# --- 1. CONFIG ---
BUCKET_NAME = "lms-reports-soral-2025"
FOLDER_NAME = "credit-reports"

# 2. GENERATE DATA
reports = []
# 10 PASSING (IDs 101-110)
for i in range(101, 111):
    reports.append({
        "id": str(i), "name": f"Qualified Customer {i}", "fico": 750,
        "annual_income": 100000, "requested_loan": 20000, # LTI 20%
        "inquiries_120d": 0, "foreclosures_24m": 0, "new_trades_24m": 0, "cc_utilization": 30
    })

# 10 FAILING (IDs 901-910) - Each fails one specific rule
scenarios = [
    {"id": "901", "requested_loan": 90000, "annual_income": 100000}, # Fail LTI (90%)
    {"id": "902", "inquiries_120d": 2},                              # Fail Inquiries
    {"id": "903", "foreclosures_24m": 1},                            # Fail Foreclosure
    {"id": "904", "new_trades_24m": 1},                               # Fail New Trade
    {"id": "905", "cc_utilization": 95},                             # Fail Utilization
    {"id": "906", "fico": 620},                                      # Fail Base FICO
    {"id": "907", "inquiries_120d": 1, "cc_utilization": 85},        # Fail Multiple
    {"id": "908", "requested_loan": 60000, "annual_income": 70000}, # Fail LTI (85%)
    {"id": "909", "new_trades_24m": 2},                               # Fail New Trade
    {"id": "910", "foreclosures_24m": 1, "fico": 800}                # Fail Foreclosure
]
reports.extend(scenarios)

# 3. UPLOAD TO GCS
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

for r in reports:
    filename = f"{r['id']}.json"
    blob = bucket.blob(f"{FOLDER_NAME}/{filename}")
    blob.upload_from_string(json.dumps(r, indent=2), content_type='application/json')

print(f"✅ Successfully uploaded 20 reports to gs://{BUCKET_NAME}/{FOLDER_NAME}/")

✅ Successfully uploaded 20 reports to gs://lms-reports-soral-2025/credit-reports/


In [ ]:
import json
import random

# CONFIGURATION FROM POLICY
prohibited_states = ["AR", "CO", "CT", "GA", "HI", "IA", "LA", "ME", "MA", "MS", "MT", "NH", "OR", "VT", "WV"]
special_apr_states = {"NY": "24.9%", "DC": "24.9%", "PA": "24.9%", "MD": "24.9%"}

# ADVERSE ACTION REASONS (PER POLICY SECTION 6)
denial_reasons = {
    "fico": "Credit score below minimum requirement of 650.",
    "income": "Income insufficient for amount of credit requested (Minimum $3,000 gross).",
    "inquiries": "Excessive personal loan inquiries in the last 120 days.",
    "utilization": "Excessive revolving credit utilization (Must be <80%).",
    "id_format": "Invalid ID format. Artha Customer IDs must be 1-4 digits only."
}



# CONFIGURATION FROM ARTHA POLICY
prohibited_states = ["AR", "CO", "CT", "GA", "HI", "IA", "LA", "ME", "MA", "MS", "MT", "NH", "OR", "VT", "WV"]
denial_reasons = {
    "fico": "Credit score below minimum requirement of 650.",
    "income": "Income insufficient for amount of credit requested (Minimum $3,000 gross).",
    "inquiries": "Excessive personal loan inquiries in the last 120 days.",
    "utilization": "Excessive revolving credit utilization (Must be <80%).",
    "foreclosure": "Foreclosure or repossession on record within the last 24 months.",
    "bankruptcy": "Bankruptcy filing within the last 7 years."
}

dataset = []

# 1. ID VALIDATION LOOP (REJECTS 5+ DIGITS)
for _ in range(20):
    valid_id = str(random.randint(1, 9999))
    dataset.append({"user": f"My ID is {valid_id}.", "model": "ID received. One step at a time: what is your gross monthly income?"})

    invalid_id = str(random.randint(10000, 99999))
    dataset.append({"user": f"My ID is {invalid_id}.", "model": "Invalid ID format. Artha Customer IDs must be between 1 and 4 digits. Please re-enter a valid ID."})

# 2. ADVERSE ACTION REASONING LOOP (SPECIFIC COMPLIANCE)
for gate, reason in denial_reasons.items():
    for _ in range(5):
        dataset.append({"user": f"Why was my application declined based on my {gate}?",
                        "model": f"[Reasoning: FCRA/ECOA Compliance] Under federal guidelines, we must provide a specific reason: {reason}"})

# 3. SINGLE-QUESTION INTERACTION LOOP (PREVENTS MULTI-INPUT CONFUSION)
for _ in range(15):
    dataset.append({"user": "I want a loan. My ID is 505 and I live in Texas.",
                    "model": "I see you're in Texas! To ensure accuracy, I only process one item at a time. Let's start with your ID: I've noted ID 505. Now, what is your gross monthly income?"})

# 4. SAFETY & COMMUNITY GUARDRAIL LOOP
safety_triggers = ["This is a scam", "I hate this bank", "You guys are terrible", "[Derogatory Language]"]
for trigger in safety_triggers:
    dataset.append({"user": trigger, "model": "I am here to assist with professional financial services only. Please keep our conversation respectful and focused on your application."})


# LOOP 2: MULTI-INPUT HANDLING (SINGLE QUESTION LOGIC)
for _ in range(15):
    dataset.append({"user": "I want a loan. My ID is 505 and I make $5,000.",
                    "model": "I've noted your ID. To ensure accuracy, I only process one item at a time. First, let's confirm: which state do you live in?"})

# LOOP 3: ADVERSE ACTION REASONS (POLICY GROUNDED)
for field, reason in denial_reasons.items():
    for _ in range(5): # Generate 5 variations for each denial type
        dataset.append({"user": f"Why was my application for {field} denied?",
                        "model": f"[Reasoning: Adverse Action Notice] Your application was declined for the following reason: {reason}"})

# LOOP 4: STATE COMPLIANCE & SPECIAL CAPS
for state in prohibited_states[:10]: # Subset for variety
    dataset.append({"user": f"I live in {state}. Can I get a loan?",
                    "model": f"No. Artha does not lend in {state}. Visit https://partners.arthafin.com for our partners."})
# Add to your existing Python loop:
loan_products = ["personal_loan", "debt_consolidation"]
other_products = ["mortgage", "auto_loan", "solar_loan"]

for _ in range(20):
    prod = random.choice(other_products)
    dataset.append({
        "user": f"I want a {prod} for $20k.",
        "model": f"{{\"product_type\": \"{prod}\", \"decision\": \"redirect\", \"message\": \"Artha only offers Personal/Debt Consolidation. Redirecting to partners.\"}}"
    })

# EXPORT TO JSONL
with open('artha_fine_tuning_2.jsonl', 'w') as f:
    for entry in dataset:
        # Standard Gemini Chat Format
        json_line = {
            "contents": [
                {"role": "user", "parts": [{"text": entry["user"]}]},
                {"role": "model", "parts": [{"text": entry["model"]}]}
            ]
        }
        f.write(json.dumps(json_line) + '\n')

print(f"✅ Generated {len(dataset)} lines of tight fintech training data.")

✅ Generated 164 lines of tight fintech training data.


In [ ]:
import gradio as gr
import vertexai
from vertexai.generative_models import GenerativeModel, Tool
# This is the correct path for Grounding and Retrieval classes
from vertexai.generative_models import grounding

In [ ]:
from google.colab import auth
auth.authenticate_user() # Required to access private dedicated endpoints

In [ ]:
{"contents": [{"role": "user", "parts": [{"text": "I have a FICO score of 620 and my monthly income is $4500. Am I eligible?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score of 620 is below our minimum requirement of 650."}]}]}

{'contents': [{'role': 'user',
   'parts': [{'text': 'I have a FICO score of 620 and my monthly income is $4500. Am I eligible?'}]},
  {'role': 'model',
   'parts': [{'text': 'Underwriting Decision: REJECTED. Reason: FICO score of 620 is below our minimum requirement of 650.'}]}]}

In [ ]:
import json

# List of your training cases
train_cases = [
    {"contents": [{"role": "user", "parts": [{"text": "FICO: 620, Income: $4500. Do I qualify?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score of 620 is below our 650 minimum."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "My FICO is 720 but I earn $2500/mo."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Monthly income of $2,500 is below our $3,000 requirement."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "FICO 680, Income $3500. Status?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet our minimum requirements (650 FICO / $3,000 income)."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Credit is 590, Income $10,000. Am I good?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score 590 does not meet our 650 minimum."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "I earn $2950 and have an 800 FICO."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Income of $2,950 is below our $3,000 requirement."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "FICO 650, Income $3000. Is this enough?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet the exact minimum thresholds."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "My score is 750 and I make $12,000/mo."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. Your profile significantly exceeds our minimum requirements."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "FICO is 649, Income $5000."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score 649 is below our mandatory 650 minimum."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "High-earner at $15k/mo but my credit is 610."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score of 610 is ineligible regardless of income."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "FICO: 710, Earnings: $3,200. Clear?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet Artha's eligibility criteria."}]}]}
]

# Write to train.jsonl
with open('train.jsonl', 'w') as f:
    for case in train_cases:
        f.write(json.dumps(case) + '\n')

print("Created train.jsonl successfully!")

Created train.jsonl successfully!


In [ ]:
val_cases = [
    {"contents": [{"role": "user", "parts": [{"text": "630 score and $6000 salary."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Minimum FICO required is 650. Your score is 630."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Income: $2800, FICO: 750."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Minimum income required is $3,000. Your income is $2,800."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Score 660, Income $3100."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet the minimum requirements."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "What if my FICO is 655 and income is $3050?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You exceed our minimum thresholds."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Score 600, Income $2000."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Both FICO and income are below requirements."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "I have 700 FICO, make $2999."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: Your income is $1 below our threshold."}]}]}, # Added comma here
    {"contents": [{"role": "user", "parts": [{"text": "Credit 651, Income $3001."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet our eligibility criteria."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Loan with 640 FICO and $8000 income?"}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: REJECTED. Reason: FICO score 640 is below the 650 requirement."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "Income $4200, FICO 675."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. You meet our requirements."}]}]},
    {"contents": [{"role": "user", "parts": [{"text": "I earn $4166/mo and have a 700 score."}]}, {"role": "model", "parts": [{"text": "Underwriting Decision: APPROVED. Both criteria qualify."}]}]}
]

# Write to validation.jsonl
import json
with open('validation.jsonl', 'w') as f:
    for case in val_cases:
        f.write(json.dumps(case) + '\n')

print("Created validation.jsonl successfully!")

Created validation.jsonl successfully!


In [ ]:
from google.colab import files

# This will trigger your browser to download the files to your 'Downloads' folder
files.download('train.jsonl')
files.download('validation.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel

# 1. Configuration
PROJECT_ID = "soral-vertex-a"
LOCATION = "us-central1"
# Get this ID from the Online Prediction > Endpoints page
ENDPOINT_ID = "527008017820942336"

# 2. Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# 3. Reference the Shared Endpoint
# Note the standard URI format: https://LOCATION-aiplatform.googleapis.com
endpoint_path = f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
model = GenerativeModel(endpoint_path)

# 4. Simple Chat Test
def artha_chat(message):
    try:
        response = model.generate_content(message)
        return response.text
    except Exception as e:
        return f"Artha Connection Error: {str(e)}"

print(artha_chat("FICO 200, Income $5000. Am I eligible?"))

Underwriting Decision: REJECTED. Reason: FICO score 200 is below our mandatory 650 minimum.


In [ ]:
import gradio as gr
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession

# 1. Configuration
PROJECT_ID = "soral-vertex-a"
LOCATION = "us-central1"
# Use your new Shared Endpoint ID here
ENDPOINT_ID = "527008017820942336"

# 2. Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# 3. Reference the Shared Endpoint as a GenerativeModel
endpoint_path = f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
model = GenerativeModel(endpoint_path)

# 4. Define the Chat Function
# message: the user's current input string
# history: the list of previous turns (Gradio handles this for you)
def predict(message, history):
    # Start a chat session using the tuned model on the endpoint
    chat = model.start_chat()

    try:
        # Send the user message to your tuned model
        response = chat.send_message(message)
        return response.text
    except Exception as e:
        return f"⚠️ Artha Connection Error: {str(e)}"

# --- CHANGE THIS ---
demo = gr.ChatInterface(
    fn=predict,
    title="Soral AI Underwriter",
    description="Custom-tuned model for loan eligibility gates.",
    # theme="soft",  <-- REMOVE THIS LINE
    examples=["FICO 720, Income $4500", "FICO 610, Income $8000"]
)

# --- ADD IT HERE ---
demo.launch(theme="soft", share=True)

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e8df432441dbb405e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [76]:
import gradio as gr
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession, GenerationConfig
from google.genai import types  # <--- This is where your configs live now

# 1. Configuration
PROJECT_ID = "soral-vertex-a"
LOCATION = "us-central1"
# Use your new Shared Endpoint ID here
ENDPOINT_ID = "3508390971140210688"

# 2. Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# 2. Updated System Instructions
SYSTEM_INSTRUCTIONS = """
# ROLE
You are Soral AI, a playful yet professional Underwriting Assistant.


# THE RULES OF THE GATE
1. NAME: Ask for Full Name. Do not proceed until you have a string that looks like a name.
2. ID: Ask for a 6-digit ID number.
   - CRITICAL: If the user provides something that is NOT 6 digits, playfully tell them "That doesn't look like a real ID!" and ask for the ID AGAIN.
   - DO NOT move to FICO until you have a valid 6-digit ID.
3. FICO: Ask for FICO score (300-850).
4. INCOME: Ask for monthly income.

# ANTI-LOOP LOGIC
- Never ask a question you have already successfully answered in the chat history.
- If the user provides "Wrong" data for the current step, STAY on that step. Do not restart from Name.
- Use the chat history to see which 'Gate' is currently open.

# INTERNAL CHECKLIST (Do this before every reply)
1. Do I have the Name? [ ]
2. Do I have the ID? [ ]
3. Do I have the FICO? [ ]
4. Do I have the Income? [ ]

Ask for the first item that is not checked. Once all are checked, provide the underwriting decision based on your policy.
"""

# 1. Define the Anti-Loop Configuration
anti_loop_config = GenerationConfig(
    temperature=0.1,       # Low but not 0; 0.1 allows slight variety to escape loops
    presence_penalty=1.5,  # Encourages the model to talk about NEW topics
    frequency_penalty=1.5, # Penalizes the model for repeating the EXACT same words
    max_output_tokens=1024, # Prevents the model from talking forever
    top_p=0.8              # Limits the word pool to the top 80% most likely words
)

# 3. Reference the Shared Endpoint as a GenerativeModel
endpoint_path = f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
model = GenerativeModel(endpoint_path,
                        system_instruction=[SYSTEM_INSTRUCTIONS],
                        generation_config=anti_loop_config # Apply the settings here
                        )


# 4. Define the Chat Function
# message: the user's current input string
# history: the list of previous turns (Gradio handles this for you)
def predict(message, history):
    # Convert Gradio history format to Gemini history format
    # This ensures the model 'remembers' it already has the name
    chat = model.start_chat(history=history)
    response = chat.send_message(message)
    try:
        # Send the user message to your tuned model
        response = chat.send_message(message)
        return response.text
    except Exception as e:
        return f"⚠️ Artha Connection Error: {str(e)}"

# --- CHANGE THIS ---
demo = gr.ChatInterface(
    fn=predict,
    title="Soral Underwriter AI",
    description="Custom-tuned model for loan eligibility gates.",
    # theme="soft",  <-- REMOVE THIS LINE
    examples=["FICO 720, Income $4500", "FICO 610, Income $8000"]
)

# --- ADD IT HERE ---
demo.launch(theme="soft", share=True)

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae37ea4172006f9ef5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
